In [95]:
import pandas as pd

 
## Описание данных

Для выполнения задания вам необходимы файлы `train.csv`, `lectures.csv`, `questions.csv`.

### train.csv
- **row_id**: (int64) ID строки.
- **timestamp**: (int64) время в миллисекундах между этим взаимодействием пользователя и первым завершенным событием этого пользователя.
- **user_id**: (int32) ID пользователя.
- **content_id**: (int16) ID контента, с которым взаимодействует пользователь.
- **content_type_id**: (int8) 0, если событие — это вопрос, заданный пользователю, 1, если событие — это лекция, которую смотрит пользователь.
- **task_container_id**: (int16) ID группы вопросов или лекций. Например, пользователь может увидеть три вопроса подряд перед тем, как увидеть объяснения. Эти три вопроса будут иметь один и тот же `task_container_id`.
- **user_answer**: (int8) ответ пользователя на вопрос, если он есть. Читайте -1 как null для лекций.
- **answered_correctly**: (int8) правильно ли ответил пользователь. Читайте -1 как null для лекций.
- **prior_question_elapsed_time**: (float32) среднее время в миллисекундах, которое потребовалось пользователю для ответа на каждый вопрос в предыдущем наборе вопросов, игнорируя любые лекции между ними. Null для первого набора вопросов или лекции пользователя.
- **prior_question_had_explanation**: (bool) видел ли пользователь объяснение и правильный ответ после ответа на предыдущий набор вопросов, игнорируя любые лекции между ними. Значение общее для всего набора вопросов и null для первого набора вопросов или лекции пользователя.

### questions.csv
Метаданные для вопросов, заданных пользователям.
- **question_id**: внешний ключ для столбца `content_id` в `train/test`, когда тип контента — вопрос (0).
- **bundle_id**: код для группы вопросов, которые подаются вместе.
- **correct_answer**: правильный ответ на вопрос. Можно сравнить с `user_answer` в `train`, чтобы проверить, был ли пользователь прав.
- **part**: соответствующий раздел теста TOEIC.
- **tags**: один или несколько детализированных кодов тегов для вопроса. Значение тегов не предоставляется, но эти коды достаточны для кластеризации вопросов.

### lectures.csv
Метаданные для лекций, которые смотрят пользователи в процессе обучения.
- **lecture_id**: внешний ключ для столбца `content_id` в `train/test`, когда тип контента — лекция (1).
- **part**: код верхнего уровня категории для лекции.
- **tag**: один код тега для лекции. Значение тегов не предоставляется, но эти коды достаточны для кластеризации лекций.
- **type_of**: краткое описание основной цели лекции.



## lectures.csv

In [96]:
lectures = pd.read_csv('C:\skilfactory\lectures.csv', sep=',')
lectures

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question
...,...,...,...,...
413,32535,8,5,solving question
414,32570,113,3,solving question
415,32604,24,6,concept
416,32625,142,2,concept


 ### Общая информация о DataFrame

In [97]:
lectures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int64 
 1   tag         418 non-null    int64 
 2   part        418 non-null    int64 
 3   type_of     418 non-null    object
dtypes: int64(3), object(1)
memory usage: 13.2+ KB


### Описательная статистика

In [98]:
lectures.describe(include='object')

,type_of
count,418
unique,4
top,concept
freq,222


In [99]:
lectures.describe()

,lecture_id,tag,part
count,418.000000,418.000000,418.000000
mean,16983.401914,94.480861,4.267943
std,9426.164660,53.586487,1.872424
min,89.000000,0.000000,1.000000
25%,9026.250000,50.250000,2.000000
50%,17161.500000,94.500000,5.000000
75%,24906.250000,140.000000,6.000000
max,32736.000000,187.000000,7.000000


В данных есть 4 уникальных типа лекций, и наиболее часто встречающийся тип - “concept”, который появляется 222 раза.

### Проверка на пропущенные значения

In [100]:
lectures.isnull().sum()

lecture_id    0
tag           0
part          0
type_of       0
dtype: int64

### Уникальные значения

In [101]:
lectures.nunique()

lecture_id    418
tag           151
part            7
type_of         4
dtype: int64

### Распределение значений

In [102]:
lectures['type_of'].value_counts()

concept             222
solving question    186
intention             7
starter               3
Name: type_of, dtype: int64

###  Вопросы распределены по различным частям теста TOEIC

In [103]:
lectures['part'].value_counts()

5    143
6     83
2     56
1     54
7     32
4     31
3     19
Name: part, dtype: int64

In [104]:
lectures['tag'].value_counts()

136    7
116    6
134    6
27     6
161    6
      ..
99     1
57     1
90     1
83     1
187    1
Name: tag, Length: 151, dtype: int64

Наиболее частые теги:
136: 7
116: 6
134: 6
27: 6
161: 6

## questions.csv

In [105]:
questions = pd.read_csv('C:\skilfactory\questions.csv', sep=',')
questions

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38
...,...,...,...,...,...
13518,13518,13518,3,5,14
13519,13519,13519,3,5,8
13520,13520,13520,2,5,73
13521,13521,13521,0,5,125


### Общая информация о DataFrame

In [106]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int64 
 1   bundle_id       13523 non-null  int64 
 2   correct_answer  13523 non-null  int64 
 3   part            13523 non-null  int64 
 4   tags            13522 non-null  object
dtypes: int64(4), object(1)
memory usage: 528.4+ KB


### Как вопросы распределены по различным частям теста TOEIC

In [107]:
questions['part'].value_counts()

5    5511
2    1647
3    1562
4    1439
6    1212
7    1160
1     992
Name: part, dtype: int64

Распределение вопросов по частям теста TOEIC:
Часть 5 содержит наибольшее количество вопросов (5511), что составляет значительную часть всего набора данных.
Часть 1 содержит наименьшее количество вопросов (992).

### Распределение правильных ответов. Это поможет понять, какие ответы чаще всего являются правильными.

In [108]:
questions['correct_answer'].value_counts()

0    3716
3    3544
1    3478
2    2785
Name: correct_answer, dtype: int64

Распределение правильных ответов:
Ответ “0” является наиболее частым правильным ответом (3716 раз)
Ответ “2” является наименее частым правильным ответом (2785 раз)

### Как вопросы распределены по различным группам (bundles)

In [109]:
questions['bundle_id'].value_counts()

7795    5
6971    5
7421    5
7770    5
8144    5
       ..
8385    1
197     1
6342    1
4295    1
0       1
Name: bundle_id, Length: 9765, dtype: int64

Распределение вопросов по группам (bundles):
Большинство групп содержат только один вопрос, что указывает на большое разнообразие групп.
Несколько групп содержат по 5 вопросов, что является максимальным количеством вопросов в одной группе.

### Распределение тегов, чтобы понять, какие теги наиболее часто встречаются

In [110]:
questions['tags'].str.split(' ', expand=True).stack().value_counts()

92     2269
38     2256
81     1969
29     1707
136    1033
       ... 
34        6
121       5
63        5
186       3
86        1
Length: 188, dtype: int64


Распределение тегов:
Теги “92” и “38” являются наиболее частыми, встречаясь 2269 и 2256 раз соответственно.
Некоторые теги встречаются крайне редко, например, тег “86” встречается только один раз.
Эти выводы могут помочь в дальнейшем анализе данных, например, в выявлении наиболее сложных частей теста или в оптимизации распределения вопросов по тесту. Если у вас есть конкретные вопросы или

## train.csv

In [111]:
# train = pd.read_csv(r'C:\skilfactory\train.csv', sep=',')
# train.head()

 - Так как прочитать весь файл на моем персональном компьютере не получилось, возму первые 1000000 строк

In [112]:
chunksize = 1000000  # количество строк в одном куске
for train in pd.read_csv(r'C:\skilfactory\train.csv', sep=',', chunksize=chunksize):
    # обработка каждого куска
    print(train.head())
    break

   row_id  timestamp  user_id  content_id  content_type_id  task_container_id  \
0       0          0      115        5692                0                  1   
1       1      56943      115        5716                0                  2   
2       2     118363      115         128                0                  0   
3       3     131167      115        7860                0                  3   
4       4     137965      115        7922                0                  4   

   user_answer  answered_correctly  prior_question_elapsed_time  \
0            3                   1                          NaN   
1            2                   1                      37000.0   
2            0                   1                      55000.0   
3            0                   1                      19000.0   
4            1                   1                      11000.0   

  prior_question_had_explanation  
0                            NaN  
1                          False  
2    

### Общая информация о DataFrame

In [113]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   row_id                          1000000 non-null  int64  
 1   timestamp                       1000000 non-null  int64  
 2   user_id                         1000000 non-null  int64  
 3   content_id                      1000000 non-null  int64  
 4   content_type_id                 1000000 non-null  int64  
 5   task_container_id               1000000 non-null  int64  
 6   user_answer                     1000000 non-null  int64  
 7   answered_correctly              1000000 non-null  int64  
 8   prior_question_elapsed_time     976277 non-null   float64
 9   prior_question_had_explanation  996184 non-null   object 
dtypes: float64(1), int64(8), object(1)
memory usage: 76.3+ MB


### Описательная статистика

In [114]:
train.describe()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time
count,1000000.000000,1.000000e+06,1.000000e+06,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,976277.000000
mean,499999.500000,7.344311e+09,1.017255e+07,5232.691523,0.019907,806.105689,1.374784,0.617504,25302.962693
std,288675.278932,1.058693e+10,6.029234e+06,3882.323419,0.139681,1025.887530,1.194168,0.525364,19710.310227
min,0.000000,0.000000e+00,1.150000e+02,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000
25%,249999.750000,5.211669e+08,4.702585e+06,2057.000000,0.000000,109.000000,0.000000,0.000000,16000.000000
50%,499999.500000,2.824811e+09,9.678259e+06,5052.000000,0.000000,391.000000,1.000000,1.000000,21000.000000
75%,749999.250000,1.010186e+10,1.556872e+07,7421.000000,0.000000,1112.000000,3.000000,1.000000,29666.000000
max,999999.000000,7.809200e+10,2.094902e+07,32736.000000,1.000000,7739.000000,3.000000,1.000000,300000.000000


 - Общее количество записей: У вас есть 1,000,000 записей в наборе данных.
 - Средние значения: Средние значения для различных столбцов, например, user_id имеет среднее значение около 10,172,550, а answered_correctly - 0.617504, что указывает на то, что примерно 61.75% ответов были правильными.
 - Стандартное отклонение: Стандартное отклонение показывает разброс данных. Например, для prior_question_elapsed_time оно составляет около 19,710, что указывает на значительную вариативность времени, затраченного на предыдущие вопросы.
 - Минимальные и максимальные значения: Минимальные и максимальные значения дают представление о диапазоне данных. Например, content_id варьируется от 0 до 32,736.
 - Квартильные значения: Квартильные значения (25%, 50%, 75%) помогают понять распределение данных. Например, 50% значений task_container_id находятся ниже 391.
 - Пропущенные значения: В столбцах prior_question_elapsed_time и prior_question_had_explanation есть пропущенные значения (23,723 и 3,816 соответственно).
 - Уникальные значения: Количество уникальных значений в различных столбцах, например, user_id имеет 3,824 уникальных значений, а content_id - 13,320.

In [115]:
train.describe(include='object')

,prior_question_had_explanation
count,996184
unique,2
top,True
freq,889887


### Проверка на пропущенные значения

In [116]:
train.isnull().sum()

row_id                                0
timestamp                             0
user_id                               0
content_id                            0
content_type_id                       0
task_container_id                     0
user_answer                           0
answered_correctly                    0
prior_question_elapsed_time       23723
prior_question_had_explanation     3816
dtype: int64

### Проверка на уникальные значения

In [117]:
train.nunique()

row_id                            1000000
timestamp                          769357
user_id                              3824
content_id                          13320
content_type_id                         2
task_container_id                    7740
user_answer                             5
answered_correctly                      3
prior_question_elapsed_time          1659
prior_question_had_explanation          2
dtype: int64

### Распределение значений


### Распределение правильных и неправильных ответов. Это поможет понять, насколько хорошо пользователи справляются с вопросами.

In [118]:
train['answered_correctly'].value_counts()

 1    637411
 0    342682
-1     19907
Name: answered_correctly, dtype: int64

Большинство пользователей справляются с вопросами правильно

### Как часто пользователи взаимодействуют с разными типами контента.

In [119]:
train['content_type_id'].value_counts()

0    980093
1     19907
Name: content_type_id, dtype: int64

Большинство взаимодействий происходит с контентом типа 0

### Какие ответы наиболее популярны

In [120]:
train['user_answer'].value_counts()

 0    278205
 1    267017
 3    257932
 2    176939
-1     19907
Name: user_answer, dtype: int64

Ответы распределены довольно равномерно, с небольшим перевесом в сторону ответа 0

### Насколько часто пользователи получают обратную связь

In [121]:
train['prior_question_had_explanation'].value_counts()

True     889887
False    106297
Name: prior_question_had_explanation, dtype: int64

Большинство пользователей получают объяснения после вопросов

### Насколько сложны вопросы

In [122]:
train['prior_question_elapsed_time'].value_counts()

17000.0     50744
16000.0     46949
18000.0     46550
19000.0     39580
15000.0     35889
            ...  
144600.0        1
254667.0        1
99333.0         1
146200.0        1
124600.0        1
Name: prior_question_elapsed_time, Length: 1659, dtype: int64

Время на предыдущий вопрос варьируется, но чаще всего оно составляет около 17,000 мс.

### Cреднее и медианное время, затраченное на предыдущие вопросы для каждого пользователя

In [123]:
user_time_stats = train.groupby('user_id')['prior_question_elapsed_time'].agg(['mean', 'median'])
print(user_time_stats)

                  mean   median
user_id                        
115       19933.311111  20000.0
124       18793.000000  21000.0
2746      18055.555556  17500.0
5382      36048.387097  25000.0
8623      26107.407407  20000.0
...                ...      ...
20913319  27666.629723  23000.0
20913864  18200.000000  16000.0
20938253  22630.590822  15000.0
20948951  19860.000000  18500.0
20949024  32757.021277  22000.0

[3824 rows x 2 columns]


### Корреляция признаков

In [124]:
train.corr()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time
row_id,1.000000,-0.000843,0.999057,-0.006783,0.004011,-0.071142,-0.002382,-0.015589,0.016242
timestamp,-0.000843,1.000000,-0.001041,0.032189,0.007441,0.364525,-0.001796,0.027571,0.016144
user_id,0.999057,-0.001041,1.000000,-0.007103,0.004102,-0.072862,-0.002361,-0.014776,0.016981
content_id,-0.006783,0.032189,-0.007103,1.000000,0.420121,0.062908,-0.110374,-0.203327,0.149778
content_type_id,0.004011,0.007441,0.004102,0.420121,1.000000,-0.013024,-0.283418,-0.438788,NaN
task_container_id,-0.071142,0.364525,-0.072862,0.062908,-0.013024,1.000000,0.012135,0.039130,-0.043670
user_answer,-0.002382,-0.001796,-0.002361,-0.110374,-0.283418,0.012135,1.000000,0.126721,0.014283
answered_correctly,-0.015589,0.027571,-0.014776,-0.203327,-0.438788,0.039130,0.126721,1.000000,0.003251
prior_question_elapsed_time,0.016242,0.016144,0.016981,0.149778,NaN,-0.043670,0.014283,0.003251,1.000000


### Корреляция prior_question_elapsed_time с content_id:
Корреляция 0.151661 указывает на слабую связь между временем, затраченным на предыдущий вопрос, и идентификатором контента. Это может означать, что некоторые типы контента требуют больше времени для ответа.

In [125]:
train[train['content_type_id'] == 0]['prior_question_elapsed_time']

0             NaN
1         37000.0
2         55000.0
3         19000.0
4         11000.0
           ...   
999995    14000.0
999996    17000.0
999997    18000.0
999998     6000.0
999999    46000.0
Name: prior_question_elapsed_time, Length: 980093, dtype: float64

In [126]:
user_time_stats = train[train['content_type_id'] == 0]['prior_question_elapsed_time'].dropna()

In [127]:
mean_time = user_time_stats.mean()
median_time = user_time_stats.median()
min_time = user_time_stats.min()
max_time = user_time_stats.max()
print(f"Mean: {mean_time}, Median: {median_time}, Min: {min_time}, Max: {max_time}")


Mean: 25302.962692965215, Median: 21000.0, Min: 0.0, Max: 300000.0


Значительное различие между минимальным и максимальным временем также подтверждает - 
НАЛИЧИЕ ВЫБРОСОВ

 ### Анализ времени ответа

In [128]:
questions = train[train['content_type_id'] == 0]

correct = questions[questions['answered_correctly'] == 1]['prior_question_elapsed_time'].mean()
incorrect = questions[questions['answered_correctly'] == 0]['prior_question_elapsed_time'].mean()
print(f'Среднее время ответа для правильных ответов: {correct}')
print(f'Среднее время ответа для неправильных ответов: {incorrect}')

Среднее время ответа для правильных ответов: 25349.965657657205
Среднее время ответа для неправильных ответов: 25215.579462489346


### Влияние лекций на результаты

In [129]:
train['is_lecture'] = train['content_type_id'] == 1
train['next_is_lecture'] = train['is_lecture'].shift(-1).fillna(False)


user_correctness = train.groupby('user_id')['answered_correctly'].mean()
lecture_effect = train.groupby('user_id').apply(lambda x: x[x['next_is_lecture']]['answered_correctly'].mean() - x[~x['next_is_lecture']]['answered_correctly'].mean())
print(lecture_effect.describe())

count    1475.000000
mean       -0.234956
std         0.329923
min        -1.054348
25%        -0.493459
50%        -0.277711
75%        -0.026844
max         0.812500
dtype: float64


 - Среднее значение влияния лекций на правильность ответов -0.234956.
В среднем студенты, которые посещают лекции, показывают результаты на 23.5% хуже по сравнению с теми, кто не посещает лекции.
 
 - Стандартное отклонение составляет 0.329923.
Влияние лекций на результаты студентов сильно варьируется.

 - Квартильные значения:
25-й процентиль: -0.493459
50-й процентиль (медиана): -0.277711
75-й процентиль: -0.026844

У большинства студентов влияние лекций на результаты отрицательное, но у некоторых студентов оно может быть положительным.

У некоторых студентов лекции могут значительно ухудшать результаты, в то время как у других они могут улучшать.
Наверно стоит проветить качество лекций

In [ ]:
user_sequences = train.groupby('user_id')['content_id'].apply(list)
print(user_sequences.head())

Последовательности контента: Анализ последовательностей контента показывает , какие лекции и материалы проходят студенты. 
Это может быть полезно для дальнейшего анализа, чтобы понять, какие конкретные лекции или материалы оказывают влияние.